Milestone 2 Group 11

Collaborators: Jill Shah & Dhruvik Patel

In [ ]:
%pip install pytube
%pip install youtube_transcript_api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.9 MB/s eta 0:00:00


In [ ]:
import os
import re
from pytube import Playlist, YouTube
from pytube.exceptions import AgeRestrictedError
from youtube_transcript_api import YouTubeTranscriptApi

def download_playlist(playlist_url, resolution):
  playlist = Playlist(playlist_url)
  playlist_name = re.sub(r'\W+', '-', playlist.title)

  if not os.path.exists(playlist_name):
    os.mkdir(playlist_name)
    for index, v in enumerate(playlist.videos[:10], start=1):
      try:
        video = YouTube(v.watch_url)
        video_resolution = video.streams.filter(res=resolution).first()
        video_filename = f"{index}. {video_resolution.default_filename}"
        video_path = os.path.join(playlist_name, video_filename)
        video_streams = video.streams.filter(res=resolution)

        if not video_streams:
          highest_resolution_stream = video.streams.get_highest_resolution()
          video_name = highest_resolution_stream.default_filename
          print(
              f"Downloading {video_name} in {highest_resolution_stream.resolution}")
          highest_resolution_stream.download(filename=video_path)


        else:
          video_stream = video_streams.first()
          video_name = video_stream.default_filename
          print(
              f"Downloading video for {video_name} in {resolution}")
          video_stream.download(filename="video.mp4")


          audio_stream = video.streams.get_audio_only()
          print(
              f"Downloading audio for {video_name}")
          audio_stream.download(filename="audio.mp4")
          os.system(
              "ffmpeg -y -i video.mp4 -i audio.mp4 -c:v copy -c:a aac final.mp4 -loglevel quiet -stats")
          os.rename("final.mp4", video_path)
          os.remove("video.mp4")
          os.remove("audio.mp4")

          srt = YouTubeTranscriptApi.get_transcript(video.video_id)
          caption_file = f"{video_path}.srt"
          with open(caption_file, 'w') as f:
            for line in srt:
              end=line["start"]+line["duration"]
              f.write(str(line["start"]) + " -- " + str(end) + "\n")
              f.write(line["text"] + "\n")

      except AgeRestrictedError:
                print(f"Video {v.title} is age-restricted and has been skipped.")
                continue


playlist_url = "https://www.youtube.com/playlist?list=PLI1yx5Z0Lrv77D_g1tvF9u3FVqnrNbCRL"
resolution = "720p"
download_playlist(playlist_url, resolution)

In [ ]:
pip install moviepy


In [ ]:
mkdir audio

In [ ]:
import os
from moviepy.editor import *

# Directory containing your videos
video_dir = '/content/Interviews'
# Directory to save audio files
audio_dir = '/content/audio'

# Ensure the audio directory exists
os.makedirs(audio_dir, exist_ok=True)

def process_video(video_path):
    # Load the video file
    video = VideoFileClip(video_path)

    # Get the audio from the video
    audio = video.audio

    # Build the audio file path
    audio_file_name = os.path.basename(video_path).replace('.mp4', '.mp3')
    audio_file_path = os.path.join(audio_dir, audio_file_name)

    # Write the audio to file
    audio.write_audiofile(audio_file_path)

# Get all video files in the video directory
video_files = [f for f in os.listdir(video_dir) if f.endswith('.mp4')]

# Process each video file
for video_file in video_files:
    video_path = os.path.join(video_dir, video_file)
    process_video(video_path)


MoviePy - Writing audio in /content/audio/2. Pink The 60 Minutes Interview.mp3


MoviePy - Done.
MoviePy - Writing audio in /content/audio/7. Attorney General Merrick Garland The 60 Minutes Interview.mp3


MoviePy - Done.
MoviePy - Writing audio in /content/audio/6. Gen Mark Milley The 60 Minutes Interview.mp3


MoviePy - Done.
MoviePy - Writing audio in /content/audio/8. Deion Sanders The 2023 60 Minutes Interview.mp3


MoviePy - Done.
MoviePy - Writing audio in /content/audio/9. Volodymyr Zelenskyy The 2023 60 Minutes Interview.mp3


MoviePy - Done.
MoviePy - Writing audio in /content/audio/10. Denzel Washington  60 Minutes Archive.mp3


MoviePy - Done.
MoviePy - Writing audio in /content/audio/1. Vice President Kamala Harris The 2023 60 Minutes Interview.mp3


MoviePy - Done.
MoviePy - Writing audio in /content/audio/5. Godfather of AI Geoffrey Hinton The 60 Minutes Interview.mp3


MoviePy - Done.
MoviePy - Writing audio in /content/audio/4. Rich Paul The 60 Minutes Interview.mp3


MoviePy - Done.


In [ ]:
! pip install git+https://github.com/openai/whisper.git


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-k5jaatg8
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-k5jaatg8
  Resolved https://github.com/openai/whisper.git to commit 1cea4357687b676b293cb5473e1ade25f5b1cef7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.7 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231106-py3-none-any.whl size=801359 sha256=5feb2573c310f86ab29420debf192ee2c6ccdb35c9cd567c820d694c4fa861f5
  Stored in directory: /tmp/pip-ephem-wheel-cache-vb4u0l13/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following depe

In [ ]:
import whisper
import os

def convert_audio_to_text_whisper_and_save(audio_directory):
    """
    This function will take the directory containing audio files, load each audio file,
    run Whisper's speech recognition on it, and save the transcribed text into a text file
    with the same name as the audio file in the same directory.
    """
    # Initialize the model
    model = whisper.load_model("base.en")

    # List all the audio files in the directory
    audio_files = [f for f in os.listdir(audio_directory) if f.endswith('.mp3')]

    # Process each audio file
    for audio_file in audio_files:
        audio_path = os.path.join(audio_directory, audio_file)
        # Load the audio file
        result = model.transcribe(audio_path)
        # Build the text file path
        text_file_path = audio_path.replace('.mp3', '.txt')
        # Write the transcription to the text file
        with open(text_file_path, 'w') as text_file:
            text_file.write(result['text'])

convert_audio_to_text_whisper_and_save('/content/audio')

100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 95.7MiB/s]


In [ ]:
%pip install pytube
%pip install youtube_transcript_api
%pip install git+https://github.com/openai/whisper.git
%pip install sacremoses
%pip install sentencepiece
%pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 677.4 kB/s eta 0:00:00
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-cg09gog4
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-cg09gog4
  Resolved https://github.com/openai/whisper.git to commit e58f28804528831904c3b6f2c0e473f346223433
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.8 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801358 sha256=272311400aa21d1dfc212250446366c2c96309ca88d4fa598e148f390fb7d125
  Stored in directory: /tmp/pip-ephem-wheel-cache-wjp9e41t/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper
ERROR: pip's dependency resolver does not currently take into account all t

In [ ]:
import os
import re
import whisper
import difflib
from pytube import Playlist
from moviepy.editor import *
from youtube_transcript_api import YouTubeTranscriptApi
from transformers import MarianMTModel, MarianTokenizer

def convert(seconds):
    seconds = seconds % 3600
    minutes = seconds / 60
    seconds %= 60
    return "%02d:%02d" % (minutes, seconds)

def download_video_and_transcript(link):
    # Loadig the playlist from the link
    playlist = Playlist(link)

    # Getting the name of the playlist
    playlist_name = re.sub(r'\W+', '-', playlist.title)

    # Making the directory to store the files
    if not os.path.exists(playlist_name):
        os.mkdir(playlist_name)

    # Making the directory to store the videos
    if not os.path.exists(f"{playlist_name}/video"):
        os.mkdir(f"{playlist_name}/video")

    # Making the directory to store the closed captions
    if not os.path.exists(f"{playlist_name}/caption"):
        os.mkdir(f"{playlist_name}/caption")

    # Loop to get the videos from the playlist
    for video in playlist.videos[:10]:
        # Skipping the age restricted videos
        if video.age_restricted:
            continue

        # Getting the video stream
        video_stream = video.streams.get_highest_resolution()

        # Getting the video name
        video_name = video_stream.default_filename

        # Downloading the video
        video_stream.download(output_path=f"{playlist_name}/video", filename=video_name)

        # Getting the video name without ".mp4" extension
        video_name = os.path.splitext(video_name)[0]

        # Getting the closed captions
        txt = YouTubeTranscriptApi.get_transcript(video.video_id)

        # Writing the closed captions into txt file
        with open(f"Interviews/caption/{video_name}.txt", 'w') as file:
            for line in txt:
                file.write(convert(line["start"]) + "\n")
                file.write(line["text"] + "\n")

download_video_and_transcript('https://www.youtube.com/playlist?list=PLI1yx5Z0Lrv77D_g1tvF9u3FVqnrNbCRL')

def extract_audio(video_path, audio_path):
    # Load the video file
    video = VideoFileClip(video_path)

    # Get the audio from the video
    audio = video.audio

    # Write the audio to file
    audio.write_audiofile(audio_path)

folder_name = "/content/Interviews"

# Making the directory to store the audios
if not os.path.exists(f"{folder_name}/audio"):
    os.mkdir(f"{folder_name}/audio")

# Getting all video files in the video directory
video_files = [f for f in os.listdir(f"{folder_name}/video") if f.endswith('.mp4')]

# Loop to get the videos from the directory
for video_file in video_files:
    # Making the path for video
    video_path = os.path.join(f"{folder_name}/video", video_file)

    # Making the path for audio
    audio_file = os.path.basename(video_path).replace('.mp4', '.mp3')
    audio_path = os.path.join(f"{folder_name}/audio", audio_file)

    # Calling the function
    extract_audio(video_path, audio_path)

def convert_audio_to_text(audio_directory):
    # Loading the Whisper model
    model = whisper.load_model("base.en")

    # Getting all the audio files from the directory
    audio_files = [f for f in os.listdir(audio_directory) if f.endswith('.mp3')]

    # Loop to get all the audios
    for audio_file in audio_files:
        # Making the path for audio
        audio_path = os.path.join(audio_directory, audio_file)

        # Transcribe the audio
        result = model.transcribe(audio_path)

        # Making the path for txt file
        text_file_path = audio_path.replace('.mp3', '.txt')
        text_file_path = text_file_path.replace('audio', 'transcript')

        # Writing the transcripts into txt file
        with open(text_file_path, 'w') as text_file:
            text_file.write(result['text'])

# Making the directory to store the audios
if not os.path.exists(f"{folder_name}/transcript"):
    os.mkdir(f"{folder_name}/transcript")

# Calling the function to transcribe
convert_audio_to_text(f"{folder_name}/audio")

def reformat(caption_path, transcript_path):
    with open(caption_path, 'r') as caption_file:
        caption_lines = caption_file.readlines()
    with open(transcript_path, 'r') as transcript_file:
        transcript_content = transcript_file.read()
    transcript_content = transcript_content.strip()
    matched_transcript = []
    transcript_content = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s', transcript_content)

    i = 1
    for line in transcript_content:
        temp = ""
        matched_transcript.append(caption_lines[i - 1][:-1])
        matched_transcript.append(line)
        temp = temp + caption_lines[i]
        ratio = difflib.SequenceMatcher(None, line, temp).ratio()
        prev = ratio
        i = i + 2
        while prev <= ratio and i < len(caption_lines):
            prev = ratio
            temp = temp + " " + caption_lines[i]
            ratio = difflib.SequenceMatcher(None, line, temp).ratio()
            i = i + 2
        i = i - 2

    with open(transcript_path, 'w') as output_file:
        for line in matched_transcript:
            output_file.write(line + '\n')

caption_files = [f for f in os.listdir(f"{folder_name}/caption") if f.endswith('.txt')]
transcript_files = [f for f in os.listdir(f"{folder_name}/transcript") if f.endswith('.txt')]

for caption_file, transcript_file in zip(caption_files, transcript_files):
    caption_path = os.path.join(f"{folder_name}/caption", caption_file)
    transcript_path = os.path.join(f"{folder_name}/transcript", transcript_file)

    reformat(caption_path, transcript_path)

from transformers import MarianMTModel, MarianTokenizer
import torch

# Load the model and tokenizer once
model_name = "Helsinki-NLP/opus-mt-en-es"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

def translate(text_path, translate_path, tokenizer, model):
    with open(text_path, 'r') as text_file, open(translate_path, 'w') as translate_file:
        for text in text_file:
            # Process each line or a small batch of lines
            inputs = tokenizer(text.strip(), return_tensors="pt", padding=True, truncation=True, max_length=512)
            translated = model.generate(**inputs)

            # Decode and write the translation
            translation = tokenizer.decode(translated[0], skip_special_tokens=True)
            translate_file.write(translation + '\n')

            # Clear memory cache (optional, can help in some scenarios)
            torch.cuda.empty_cache()

# Usage
#translate(text_path, translate_path, tokenizer, model)
# Making the directory to store the translations
if not os.path.exists(f"{folder_name}/translated"):
    os.mkdir(f"{folder_name}/translated")

# Getting all captions files in the audio directory
text_files = [f for f in os.listdir(f"{folder_name}/transcript") if f.endswith('.txt')]

# Loop to get the captions from the directory
for text_file in text_files:
    # Making the path for caption
    text_path = os.path.join(f"{folder_name}/transcript", text_file)

    # Making the path for translation
    translate_path = os.path.join(f"{folder_name}/translated", text_file)

    print(translate_path)

    # Calling the function
    translate(text_path, translate_path, tokenizer, model)

MoviePy - Writing audio in /content/Interviews/audio/Deion Sanders The 2023 60 Minutes Interview.mp3


MoviePy - Done.
MoviePy - Writing audio in /content/Interviews/audio/Godfather of AI Geoffrey Hinton The 60 Minutes Interview.mp3


MoviePy - Done.
MoviePy - Writing audio in /content/Interviews/audio/Denzel Washington  60 Minutes Archive.mp3


MoviePy - Done.
MoviePy - Writing audio in /content/Interviews/audio/Attorney General Merrick Garland The 60 Minutes Interview.mp3


MoviePy - Done.
MoviePy - Writing audio in /content/Interviews/audio/Volodymyr Zelenskyy The 2023 60 Minutes Interview.mp3


...

vocab.json:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

/content/Interviews/translated/Denzel Washington  60 Minutes Archive.txt
/content/Interviews/translated/Pink The 60 Minutes Interview.txt
/content/Interviews/translated/Deion Sanders The 2023 60 Minutes Interview.txt
/content/Interviews/translated/Volodymyr Zelenskyy The 2023 60 Minutes Interview.txt
/content/Interviews/translated/Rich Paul The 60 Minutes Interview.txt
/content/Interviews/translated/Attorney General Merrick Garland The 60 Minutes Interview.txt
/content/Interviews/translated/Gen Mark Milley The 60 Minutes Interview.txt
/content/Interviews/translated/Godfather of AI Geoffrey Hinton The 60 Minutes Interview.txt
/content/Interviews/translated/Vice President Kamala Harris The 2023 60 Minutes Interview.txt
